# Distributed Shared Memory

We usually assume processes pass messages via the network.
If we could store messages in a shared memory location, we could be better people.

While a multi-processor system could use its physically shared memory as a common buffer, distributed systems will instead have to have a common *remote* buffer.

## Invalidate Approach
### Properties
* Each process maintains a cache, which stores pages recently accessed by that process.
* Memory pages are mapped in local memory, as well as the cache
* R/W first hits the cache
    * If present: page *hit*
    * If missing: page *fault* (kernel trap)
        * Invoke DSM software to retrieve page from another process on network. Probably use multicast, everyone else does.
* Owner = process with latest version of page
* Page is either in R or W state
* When in R,
    * Owner has a R copy in its local cache
    * Other processes may also have R copies
    * No W copies exist
* When in W state,
    * Only owner has a copy (exclusive)
    
### Scenarios
#### Reading
* You can use your cached copy if
    * P1 is owner of page in R state
        * Read from cache
    * P1 is owner of page in W state
        * Read from cache
    * P1 is owner of page in R state. Other processes also have page in R state
        * Read from cache
    * P1 has page in R state
        * Other processes also have page in R state
        * Some other process is owner
        * Read from cache
* You need to talk to people (processes) if
    * P1 wants to R; no copy; other process has R copy
        * P1 asks for a copy of the page (multicast)
        * Owner returns copy of page
        * P1 marks page as in R state
    * P1 wants to R; no copy; other process has W copy
        * P1 locates owner's copy via multicast
        * Owner degrades copy from W to R; sends to P1
        * P1 marks page as R
#### Writing
* You *don't* have to talk to people if
    * P1 is owner of W copy
        * Write to cache
* You *do* have to talk to people if
    * P1 is owner of R copy; other processes have R copy
        * P1 asks other processes to invalidate their copy
        * Mark page as W
        * Write to page
    * P1 P1 has R copy; other process is owner
        * P1 requests other processes to invalidate their copies
        * P1 receives ownership copy of page
        * P1 marks page as W
        * Writes
    * P1 has no copy
        * Request other proceses to invalidate their copies
        * Receive ownership copy of page
        * mark page as W
        * writes
### Downsides
* Flip-flopping invalidations between processes
* Lots of network transfers of files
* Can happen when unrelated variables fall on same page
    * False sharing - different processes have variables on the same page, causing much flip-flopping
* __locality of interest__ section of page a process cares about

## Update Approach
* Multiple processes can have W state pages
* On a write to a page, multicast newly written value (or part of page) to all other holders
* Preferrable to Invalidate when:
    * Heavy sharing between processes
    * Writes are small
    * Page sizes are large
    
* Invalidate is usually better in the real-world
        
## Consistency
* Whenver multiple processes are sharing data, you have a question of consistency across all system